In [ ]:
!pip install mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.7/229.7 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00


In [ ]:
!pip install tqdm

In [ ]:
import pandas as pd
import time
import ast

df = pd.read_csv('questions.csv')

print(df.head(5))


   id                                           question  \
0   0  Devant un exanthème roséoliforme fébrile de l'...   
1   1  A propos de l’insuffisance cardiaque, quelle(s...   
2   2  Quelle(s) est (sont) la (les) réponse(s) vraie...   
3   3  Une patiente de 58 ans  vient aux urgences de ...   
4   4  Un patient de 55 ans présentant un hyperlympho...   

                                            answer_A  \
0                                 Un exanthème subit   
1  L’auscultation cardiaque peut mettre en éviden...   
2                      Drainage bronchique quotidien   
3                      Vous hospitalisez la patiente   
4                                              Aucun   

                                            answer_B  \
0                        Un mégalérythème épidémique   
1  L’auscultation cardiaque peut mettre en éviden...   
2                         Corticothérapie systémique   
3     Vous mettez en place un remplissage vasculaire   
4                     

In [ ]:
import os
from mistralai import Mistral

api_key = "4LqG6HUTNimw0dUX9ZbJBYbjH0BpVSEJ"
client = Mistral(api_key=api_key)

comprehension_id = "ag:c39d428d:20241012:comprehension:39a4887b"
expert_ids = {
        1 : "ag:c39d428d:20241012:expert-1:573e3bd3",
        "categoriseur" :"ag:c39d428d:20241013:categoriseur:1de004b9" ,
        "biochimie" : "ag:c39d428d:20241012:expert-5:81fa4ef0",
        "anatomie" : "ag:c39d428d:20241012:expert-2:d5bb9a23",
        "physiologie" : "ag:c39d428d:20241012:expert-3:f17c3834",
        "pathologie" : "ag:c39d428d:20241012:expert-pathologie:4b143d73",
        "bacteriologie" : "ag:c39d428d:20241012:expert-bacteriologie:f39b3442",
        "chirurgie" : "ag:c39d428d:20241013:expert-chirurgie:a9ffed1f",
        "pediatrie" : "ag:c39d428d:20241013:expert-pediatrie:9139b7f5",
        "immunologie" : "ag:c39d428d:20241013:expert-immunologie:3be69ccd",
        "psychiatrie" : "ag:c39d428d:20241013:expert-psychiatrie:bf0cddd2",
        "radiologie" : "ag:c39d428d:20241013:expert-radiologie:71eb84f8",
        "medecine legale" : "ag:c39d428d:20241013:expert-medecine-legale:2104916a",
        "gynecologie" : "ag:c39d428d:20241013:expert-gynecologie:3bf13fbd",
        "nephrologie" : "ag:c39d428d:20241013:expert-nephrologie:104c588c",
        "epidemiologie" : "ag:c39d428d:20241013:expert-epidemiologie:b8c48f81",
        "physiopathologie" : "ag:c39d428d:20241013:expert-physiopathologie:02f459f1",
        "synthetiseur" : "ag:c39d428d:20241013:synthetizeur:b18484ed"

}


In [ ]:
def get_row_by_index(df, index):
    if index < 0 or index >= len(df):
        return None
    else :
      row = df.iloc[index]
      question = row['question']
      answer_A = row['answer_A']
      answer_B = row['answer_B']
      answer_C = row['answer_C']
      answer_D = row['answer_D']
      answer_E = row['answer_E']

      return f"Question: {question} | Answer A: {answer_A}, Answer B: {answer_B}, Answer C: {answer_C}, Answer D: {answer_D}, Answer E: {answer_E}"


In [ ]:
def run_comprehension(row):


    query = get_row_by_index(df, row)


    try:
        response = client.agents.complete(
            agent_id= "ag:c39d428d:20241012:comprehension:39a4887b",
            messages = [
                {
                    "role": "user",
                    "content":  query
                },
            ]
        )
        result = response.choices[0].message.content
        return query + "\n" + result
    except Exception as e:
        print(f"Request failed: {e}. Please check your request.")
        return None


In [ ]:
def run_expert(query, expert_name):


    try:
        response = client.agents.complete(
            agent_id= expert_ids[expert_name],
            messages = [
                {
                    "role": "user",
                    "content":  query
                },
            ]
        )
        result = response.choices[0].message.content
        return result
    except Exception as e:
        print(f"Request failed: {e}. Please check your request.")
        return None




In [ ]:

def test(row):

  comprehension = run_comprehension(row)
  final_query = get_row_by_index(df, row) + "\n" + comprehension
  result = run_expert(final_query, 1)

  return result , final_query



In [ ]:
def expert_domains(query_00,query_01) :
   # Create a list of lists to store both i and its value
  experts_answers = []
  for i in query_01:
      value = run_expert(query_00, i)
      experts_answers.append([i, value])  # Store i and value as a sublist
      time.sleep(1.5)

  return experts_answers


# Run Pipeline

In [ ]:
def run_pipeline_test(row):
  # Comprehension
  query_00 = str(run_comprehension(row))

  #Categoriseur
  query_01 = ast.literal_eval(run_expert(query_00, 'categoriseur'))
  time.sleep(1.2)
  #Experts
  expert_answers = str(expert_domains(query_00, query_01))
  time.sleep(1.2)
  #Synthese
  answer = str(run_expert(expert_answers, 'synthetiseur'))

  return answer



test

In [ ]:
print(run_pipeline_test(90))

B,D


In [157]:
from tqdm.notebook import tqdm

# Initialize the results DataFrame
results_df = pd.DataFrame(columns=['answer'])

# Use tqdm to add a progress bar while iterating over the DataFrame's index
for index in tqdm(df.index, desc="Processing Rows"):
    try:
        result = run_pipeline_test(index)
        time.sleep(2)
        if result:
            results_df = pd.concat([results_df, pd.DataFrame({'Result': [result]})], ignore_index=True)
    except Exception as e:
        print(f"Error processing row {index}: {e}")

# Display the final results DataFrame
print(results_df)


Processing Rows:   0%|          | 0/103 [00:00<?, ?it/s]

Request failed: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}. Please check your request.
Request failed: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}. Please check your request.
Error processing row 17: malformed node or string: None


KeyboardInterrupt: 